In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from tqdm import tqdm

# Funções Auxiliares

In [ ]:
def start(results, confidence=0.5):
    if results.pose_landmarks and results.face_landmarks:
        visibility = sum([landmark.visibility for landmark in results.pose_landmarks.landmark[:25]])/25
        if visibility > confidence:
            if results.right_hand_landmarks or results.left_hand_landmarks:
                rh_line = (results.pose_landmarks.landmark[18].y + results.pose_landmarks.landmark[20].y + results.pose_landmarks.landmark[22].y)/3
                lh_line = (results.pose_landmarks.landmark[17].y + results.pose_landmarks.landmark[19].y + results.pose_landmarks.landmark[21].y)/3
                hip_line = (results.pose_landmarks.landmark[23].y + results.pose_landmarks.landmark[24].y)/2
                if hip_line > rh_line or hip_line > lh_line:
                    return True
        else:
            print(f'Baixa visiblidade do pose: {visibility}')
    return False

def get_landmarks(results, fclms=[10, 67, 297, 54, 284, 162, 389, 234, 454, 132, 361, 58, 288, 136, 365, 149, 378, 148, 377]):
    landmarks = {}
    for i, landmark in enumerate(results.pose_landmarks.landmark[:25]):
        landmarks[f'PS{i}x'] = landmark.x
        landmarks[f'PS{i}y'] = landmark.y
    for i, landmark in enumerate([results.face_landmarks.landmark[fclm] for fclm in fclms]):
        landmarks[f'FC{i}x'] = landmark.x
        landmarks[f'FC{i}y'] = landmark.y
    if results.right_hand_landmarks:
        for i, landmark in enumerate(results.right_hand_landmarks.landmark):
            landmarks[f'RH{i}x'] = landmark.x
            landmarks[f'RH{i}y'] = landmark.y
    else:
        for i in range(21):
            landmarks[f'RH{i}x'] = None
            landmarks[f'RH{i}y'] = None
    if results.left_hand_landmarks:
        for i, landmark in enumerate(results.left_hand_landmarks.landmark):
            landmarks[f'LH{i}x'] = landmark.x
            landmarks[f'LH{i}y'] = landmark.y
    else:
        for i in range(21):
            landmarks[f'LH{i}x'] = None
            landmarks[f'LH{i}y'] = None
    if len(landmarks) != (134+(len(fclms)*2)):
        raise ValueError(f'Erro na extração dos landmarks: {len(landmarks)}')
    else:
        return landmarks

def centralize(landmarks:dict):
    center = {}
    center['x'] = (landmarks['PS0x'] + landmarks['PS9x'] + landmarks['PS10x'])/3 - 0.5
    center['y'] = (landmarks['PS0y'] + landmarks['PS9y'] + landmarks['PS10y'])/3 - 0.5
    for lm in landmarks:
        if landmarks[lm]:
            if 'x' in lm:
                landmarks[lm] -= center['x']
            elif 'y' in lm:
                landmarks[lm] -= center['y']
            else:
                raise ValueError(f'Nome de landmark inválido: {lm}')
    return landmarks

def cleanse(map_landmarks, threshold=0.8):
    df = pd.DataFrame(map_landmarks)
    for col in df.columns:
        if df[col].isnull().mean() >= threshold:
            df[col] = None
    df = df.astype(float).interpolate(method='linear').ffill().bfill()
    return df

# Função de Mapeamento

In [ ]:
def mapping(path:str, resize:int=1, confidence=0.7):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise ValueError("Erro ao abrir o vídeo.")
    holistic = mp.solutions.holistic.Holistic(static_image_mode=True, model_complexity=2, smooth_landmarks=False, min_detection_confidence=confidence, min_tracking_confidence=confidence)
    map_landmarks = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        height, width = frame.shape[:2]
        frame = cv2.resize(frame, (width//resize, height//resize))
        results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if start(results):
            landmarks = get_landmarks(results)
            map_landmarks.append(centralize(landmarks))
    holistic.close()
    cap.release()
    cv2.destroyAllWindows()
    return cleanse(map_landmarks)

In [ ]:
INPUT_PATH = 'DATASET/VIDEOS'
OUTPUT_PATH = 'DATASET/LANDMARKS'

mapping(f'{INPUT_PATH}/Sapo1.mp4')

,PS0x,PS0y,PS1x,PS1y,PS2x,PS2y,PS3x,PS3y,PS4x,PS4y,...,LH16x,LH16y,LH17x,LH17y,LH18x,LH18y,LH19x,LH19y,LH20x,LH20y
0,0.502190,0.463176,0.517785,0.431170,0.530266,0.438079,0.541009,0.445819,0.486849,0.422507,...,0.479963,1.209447,0.474678,1.185431,0.471550,1.229454,0.480130,1.233665,0.487086,1.226283
1,0.502189,0.465419,0.517905,0.431978,0.530250,0.438571,0.539389,0.445801,0.487210,0.424831,...,0.467649,1.139169,0.463409,1.128656,0.463075,1.174176,0.474918,1.167345,0.479275,1.149386
2,0.502580,0.467584,0.518233,0.434979,0.529222,0.441593,0.539105,0.448729,0.487505,0.426177,...,0.458748,1.087587,0.449386,1.064463,0.451693,1.115601,0.464042,1.109771,0.469121,1.092980
3,0.502297,0.466956,0.517806,0.434486,0.528973,0.441123,0.538936,0.448451,0.487194,0.425756,...,0.457459,1.084925,0.451489,1.067697,0.452419,1.114776,0.463611,1.109785,0.468043,1.093676
4,0.502508,0.468308,0.517280,0.435379,0.528692,0.441402,0.538347,0.448033,0.487856,0.429191,...,0.452043,1.049592,0.443648,1.025721,0.446084,1.079786,0.458203,1.072905,0.461930,1.054525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.502693,0.466675,0.518344,0.433411,0.530165,0.438147,0.538252,0.442496,0.488573,0.423814,...,0.408378,1.102314,0.401777,1.066984,0.405264,1.113609,0.414693,1.112017,0.418803,1.099801
78,0.501282,0.466949,0.516322,0.433481,0.528593,0.439393,0.538260,0.445825,0.485884,0.427503,...,0.419380,1.176045,0.420750,1.109975,0.418020,1.153839,0.422557,1.171712,0.428337,1.181097
79,0.503989,0.468298,0.519461,0.435184,0.529446,0.441477,0.537692,0.445956,0.488357,0.422776,...,0.430015,1.185708,0.438483,1.153439,0.426192,1.189449,0.432353,1.195664,0.440312,1.191030
80,0.504096,0.468106,0.519635,0.434806,0.529391,0.441222,0.537811,0.445689,0.488449,0.422705,...,0.433869,1.190706,0.442406,1.172772,0.431078,1.208789,0.438352,1.210614,0.445767,1.200610


# Salvando

In [ ]:
INPUT_PATH = 'DATASET/VIDEOS'
OUTPUT_PATH = 'DATASET/LANDMARKS'
for video in tqdm(os.listdir(INPUT_PATH), total=len(os.listdir(INPUT_PATH)), desc='SAVING', ncols=100):
    landmarks = mapping(f'{INPUT_PATH}/{video}')
    landmarks.to_parquet(f'{OUTPUT_PATH}/{video.replace('mp4', 'parquet')}')